In [ ]:
from dove.utils.vcf import Vcf

In [ ]:
def vcf_to_df(vcf_file):
    with Vcf(vcf_file) as f:
        f.vdf['CSQ'] = f.get_variant_info(concat=True, drop=True)['CSQ']
        f.vdf = f.get_sample_format(concat=True, drop=True)
        vep_header = [i for i in f.meta_info['INFO'] if 'CSQ' in i][0].split('Format: ')[
            1].strip('">').split('|')
        f.vdf.CSQ = f.vdf.CSQ.str.split(',')
        df = f.vdf.explode('CSQ')

    df[vep_header] = df.CSQ.str.split('|', expand=True)
    df.drop(['INFO', 'CSQ'], axis=1, inplace=True)
    df = df[df.PICK == "1"]
    df['GT'] = df['GT'].replace({'0/1': 'Het', '1/1': 'Hom'})
    df['PUBMED'] = df['PUBMED'].str.replace('&', ' ')
    if 'PHENOTYPES' in list(df):
        df['PHENOTYPES'] = df['PHENOTYPES'].str.replace('__', ';').str.replace('&', ' & ').str.replace('_', ' ')
    return df

In [ ]:
df = vcf_to_df("file.vcf.gz")